## Textrank

Загрузка текстов:

In [ ]:
!wget https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json

Установка необходимых библиотек:

In [ ]:
!pip install summa rouge

In [ ]:
!pip install git+https://github.com/Koziev/rutokenizer

  Cloning https://github.com/Koziev/rutokenizer to /tmp/pip-req-build-gjbf23gy
  Running command git clone -q https://github.com/Koziev/rutokenizer /tmp/pip-req-build-gjbf23gy
  Created wheel for rutokenizer: filename=rutokenizer-0.0.24-cp37-none-any.whl size=30370082 sha256=e82922f62d39d0af503b7e37562f2c44bfa88b3baef9276ee8f1bbe2a40033b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-4v0j0van/wheels/a6/66/ef/7c52e143b099c8aeaaf3bccc4640c065b87d7cbfb13066005d
Successfully built rutokenizer


In [ ]:
import json
import nltk
import rutokenizer
import networkx as nx
from rouge import Rouge
from nltk import sent_tokenize
from tqdm.notebook import tqdm
from itertools import combinations

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open ('judgements_test.json', encoding='utf-8') as f:
  data_test = json.load(f)

In [ ]:
t = rutokenizer.Tokenizer()
t.load()

За основу были взяты материалы семинаров Школы глубокого обучения на базе ФПМИ МФТИ (https://www.dlschool.org)

In [ ]:
def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    ''' 
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2)) / (len(words1) + len(words2))


def gen_text_rank_summary(text, calc_similarity=unique_words_similarity):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = sent_tokenize(text)
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [t.tokenize(sentence) for sentence in sentences]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = 3
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])
    
    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    return predicted_summary

Подсчет метрики:

In [ ]:
rouge = Rouge()

In [ ]:
references = []
predictions = []

for item in tqdm(data_test.keys()):
  predicted = gen_text_rank_summary(data_test[item]['text']) 
  predictions.append(predicted)
  references.append(data_test[item]['summary'])

scores = rouge.get_scores(predictions, references, avg=True)

In [ ]:
scores

{'rouge-1': {'f': 0.13746881307082262,
  'p': 0.12168137845395724,
  'r': 0.18411925626100764},
 'rouge-2': {'f': 0.04295564635446474,
  'p': 0.03683887541914653,
  'r': 0.05932368313531569},
 'rouge-l': {'f': 0.12958941952588346,
  'p': 0.12390410716321869,
  'r': 0.15522943260138733}}

Примеры summary:

In [ ]:
predictions[0]

'Ответчик представил отзыв, который приобщен к материалам дела в порядке  Арбитражного процессуального кодекса Российской Федерации. Проверив в соответствии со , , ,  Арбитражного процессуального кодекса Российской Федерации законность принятых судебных актов, обоснованность доводов, изложенных в кассационной жалобе, суд кассационной инстанции не находит оснований для отмены обжалуемых судебных актов ввиду следующего. Отказывая в удовлетворении иска, суды пришли к выводу о том, что по требованиям о взыскании излишне уплаченной арендной платы и обеспечительного платежа в связи с заключением соглашения о расторжении договора от 31.05.2016, срок исковой давности истек 15.06.2019, при этом иск направлен в суд 16.08.2019, то есть за пределами срока исковой давности.'

In [ ]:
references[0]

'Рассматривается дело o взыскании суммы обеспечительного платежа, излишне уплаченной арендной платы в соответствии с соглашением о расторжении договора аренды нежилого помещения, процентов за пользование чужими денежными средствами.  Ответчиком обязательства по соглашению не исполнены.  В удовлетворении требования отказано, поскольку пропущен срок исковой давности.'

In [ ]:
predictions[999]

'Представитель истца в заседании суда против доводов кассационной жалобы возражал, в том числе и по мотивам, изложенным в отзыве. Подрядчик обязался выполнить работы, указанные в п. При таких обстоятельствах, суд в обжалуемых актах, по мнению кассационной коллегии, правильно удовлетворил требования истца лишь в части взыскания неустойки в сумме 1.000.000 руб.'

In [ ]:
references[999]

'Заявляется требование o взыскании неустойки.  Сторонами был заключен договор подряда на выполнение строительно-монтажных работ по реконструкции теплового ввода.  Требование удовлетворено в части, поскольку ответчик нарушил установленные договором сроки выполнения работ. Размер неустойки снижен на основании ст. 333 ГК РФ.'